## Chatbot com resumo de mensagens e memória armazena em Banco de Dados externo

checkpointers mais avançados que suportam bancos de dados externos.

Como usar o SQLite como um checkpointer, mas outros checkpointers, como o PostgreSQL, estão disponíveis!

### SQLite

Um bom ponto de partida é o checkpointer SqliteSaver.

O SQLite é um banco de dados SQL pequeno, rápido e muito popular.

Se fornecermos ":memory:", ele cria um banco de dados SQLite em memória.
 
**Desa forma fica localmente na máquina**

A diferença entre usar sqlite3.connect(db_path, ...) e sqlite3.connect(":memory:") reside no local onde os dados do banco de dados SQLite são armazenados e, consequentemente, na sua persistência e acessibilidade.


1. Armazenamento em Arquivo (Persistente)
~~~Python

db_path = "state_db/example.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
~~~

* Onde Salva: O banco de dados é armazenado em um arquivo físico no disco rígido ou SSD do seu computador (no caminho especificado por db_path, ex: state_db/example.db).

* Persistência: Os dados são persistentes. Isso significa que, mesmo se o seu programa Python parar, o computador for desligado, ou você fechar a conexão, os dados estarão intactos no arquivo example.db e podem ser lidos na próxima vez que você se conectar.

* Uso: Ideal para dados de produção, logs, histórico de conversas (como no LangGraph com checkpoints), ou qualquer informação que precise sobreviver ao término do programa.

* Observação sobre check_same_thread=False: Este parâmetro é específico para permitir que a conexão seja usada por múltiplas threads, mas não altera a persistência dos dados no disco.

2. Armazenamento In-Memory (Volátil)

~~~Python

conn = sqlite3.connect(":memory:")
~~~

* Onde Salva: O banco de dados é criado e armazenado diretamente na Memória de Acesso Aleatório (RAM) do computador.

* Persistência: Os dados são voláteis. O banco de dados só existe enquanto a conexão (conn) estiver aberta. Assim que a conexão for fechada (conn.close()) ou o programa terminar, todos os dados são perdidos irrecuperavelmente.

* Uso: Ideal para:

* Testes: Criação rápida de um banco de dados limpo para testes unitários.

* Cache Temporário: Armazenar dados intermediários que só são necessários durante a execução do programa.

* Desempenho: Oferece o desempenho mais rápido possível, pois a leitura e escrita ocorrem na RAM, evitando a latência do disco.

In [ ]:
import sqlite3

# In memory Memória de Curto Prazo (RAM)
conn= sqlite3.connect(":memory:")

Mas, se fornecermos o caminho do banco de dados, ele criará um banco de dados para nós!

O parâmetro check_same_thread=False no método sqlite3.connect() serve para desativar a verificação de segurança que o módulo sqlite3 do Python usa por padrão.

🔒 A Regra Padrão (check_same_thread=True)
Por padrão, o sqlite3 aplica uma medida de segurança rígida:

Thread de Criação: Quando você cria um objeto de conexão (conn), o Python registra a thread (linha de execução) que o criou.

Restrição: O objeto conn e quaisquer objetos relacionados a ele (como cursors) só podem ser usados por essa mesma thread que o criou.

Objetivo: Isso evita condições de corrida (race conditions) e corrupção de dados que poderiam ocorrer se múltiplas threads tentassem acessar e modificar o mesmo banco de dados SQLite simultaneamente, já que o SQLite (por padrão) não é otimizado para acesso concorrente em multithreading.

O que Acontece se Você Violar: Se outra thread tentar usar a conexão, você receberá um erro: sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread.

In [ ]:
# Salvando em db local
db_path= "state_db/example.db"
conn= sqlite3.connect(db_path, check_same_thread= False)

In [ ]:
# Checkpointer
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)


In [ ]:
from typing_extensions import Literal
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState


model = ChatOpenAI(model="gpt-4o",temperature=0)

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State):
    
    # Get summary if it exists
    summary = state.get("summary", "")

    # If there is summary, then we add it
    if summary:
        
        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]
    
    else:
        messages = state["messages"]
    
    response = model.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State):
    
    # First, we get any existing summary
    summary = state.get("summary", "")

    # Create our summarization prompt 
    if summary:
        
        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
        
    else:
        summary_message = "Create a summary of the conversation above:"

    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    
    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State)-> Literal ["summarize_conversation",END]:
    
    """Return the next node to execute."""
    
    messages = state["messages"]
    
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    
    # Otherwise we can just end
    return END

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START

# Define a new graph
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile
graph = workflow.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Create a thread
config = {"configurable": {"thread_id": "1"}}

# Start conversation
input_message = HumanMessage(content="hi! I'm Lance")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="what's my name?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="i like the 49ers!")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "1"}}
graph_state = graph.get_state(config)
graph_state